In [2]:
%run ../utils/commonImports.py
%run ../utils/tradingImports.py
%matplotlib inline

# Load Data

In [3]:
def calculate_EMA(history, col, time_period):
    return calculateTalib('EMA', {col: history[col].values}, {'timeperiod': time_period})


def add_EMA(df):
    df['blue'] = calculate_EMA(df, 'close', 8)
    df['green'] = calculate_EMA(df, 'close', 13)
    df['yellow'] = calculate_EMA(df, 'close', 21)
    df['red'] = calculate_EMA(df, 'close', 55)
    return df

In [4]:
def load_and_divide(db, table, from_date=None, to_date=None, train_ratio=0.7):
    data = load_trading_data(db, table, from_date, to_date)
    data = add_EMA(data)
    data_train, data_test = divide_train_and_test(data, train_ratio)
    return data, data_train, data_test


db = 'D:\\Dropbox\\My work\\krypl-project\\sqlite\\ploniex-chart-data\\USDT_BTC.db'
data, data_train, data_test = load_and_divide(db, 'chart_data', from_date='2017-01-01')
ohlc, ohlc_train, ohlc_test = data.copy(), data_train.copy(), data_test.copy()


features = ['red', 'yellow', 'green', 'blue']
data_manager_train = CurrencyDataManager(data_train['close'], data_train[features])
data_manager_test = CurrencyDataManager(data_test['close'], data_test[features])
wallet = {'usdt': 1000}
contract_pair = ContractPair.new('usdt', 'btc')

# Explore

In [ ]:
correct_pattern = ohlc_train.query("red < yellow < green < blue").copy()

In [ ]:
correct_pattern.shape

In [ ]:
def ratio(df, c1, c2):
    return (df[c1] / df[c2]) - 1


def distplot(df, col, x_max):
    plt.figure(figsize=[12, 6])
    plt.title(col, fontproperties=title_font)
    sns.distplot(df[col])
    plt.xlim([0, x_max])

correct_pattern['ratioRY'] = ratio(correct_pattern, 'yellow', 'red')
correct_pattern['ratioYG'] = ratio(correct_pattern, 'green', 'yellow')
correct_pattern['ratioGB'] = ratio(correct_pattern, 'blue', 'green')

In [ ]:
distplot(correct_pattern, 'ratioRY', 0.015)
distplot(correct_pattern, 'ratioYG', 0.005)
distplot(correct_pattern, 'ratioGB', 0.005)

# Implementation

In [ ]:
from modeling.strategy import Strategy


class CustomStrategy(Strategy):
    def __init__(self, exchange, data_manager, contract_pair, trade_size, willing_loss, target_profit,
                threshold_ry, threshold_yg, threshold_gb):
        super().__init__(exchange, data_manager, contract_pair, trade_size, willing_loss, target_profit)
        self.history_len = 1
        self.threshold_ry = threshold_ry
        self.threshold_yg = threshold_yg
        self.threshold_gb = threshold_gb
    
    def has_correct_order(self, red, yellow, green, blue):
        def ratio(x, y):
            return (x / y) - 1
        
        return ratio(yellow, red) >= self.threshold_ry \
            and ratio(green, yellow) >= self.threshold_yg \
            and ratio(blue, green) >= self.threshold_gb \
    
    @staticmethod
    def is_pattern_destroyed(red, yellow, green, blue):
        return (red > yellow) or (red > green) or (red > blue)
    
    def trade(self):
        
        in_row = 0
        after_sell = False
        while self.data_manager.has_tick():            
            history, price = self.data_manager.tick(self.history_len)
            
            if history.shape[0] == 0:
                continue
            
            red, yellow, green, blue = history[0]
            if np.isnan(red):
                continue

            if self.has_correct_order(red, yellow, green, blue):
                in_row += 1
            else:
                in_row = 0
                after_sell = False
            
            if not self.opened and not after_sell and in_row > 1:
                self.buy(price)
                price_bought = price
            elif self.opened and self.is_risky(price_bought, price):
                self.sell_all(price)
                after_sell = True
            elif self.opened and self.is_target_satisfied(price_bought, price):
                self.sell_all(price)
                after_sell = True
            elif self.opened and self.is_pattern_destroyed(red, yellow, green, blue):
                self.sell_all(price)
                after_sell = True
        
        if self.opened:
            self.sell_all(price)


# Learning

In [ ]:
def create_train_strategy(threshold_ry, threshold_yg, threshold_gb):
    data_manager = deepcopy(data_manager_train)
    exchange = BackTestExchange(data_manager, deepcopy(wallet), 0.0025)
    strategy = CustomStrategy(exchange, data_manager, contract_pair, 100, willing_loss=0.5, target_profit=0.05,
                             threshold_ry=threshold_ry, threshold_yg=threshold_yg, threshold_gb=threshold_gb)
    return data_manager, exchange, strategy

In [ ]:
data_manager, exchange, strategy = create_train_strategy(0.005, 0.001, 0.0015)
strategy.trade()
strategy.stats('usdt').report()

## Learn Thresholds

In [ ]:
s['totalProfit']

In [ ]:
import timeit


def stats_row(strategy, ry, yg, gb):
    stats = strategy.stats('usdt').report()
    stats = stats.transpose()
    stats['threshold_ry'] = ry
    stats['threshold_yg'] = yg
    stats['threshold_gb'] = gb
    return stats.reset_index().drop('index', axis=1) 

def optimize(rys, ygs, gbs):
    stats = pd.DataFrame()
    for ry in rys:
        for yg in ygs:
            for gb in gbs:
                print('ry = ', ry, 'yg = ', yg, 'gb = ', gb, end='\t')
                start = timeit.timeit()
                data_manager, exchange, strategy = create_train_strategy(ry, yg, gb)
                strategy.trade()
                row_stats = stats_row(strategy, ry, yg, gb)
                stats = stats.append(row_stats)
                profit = row_stats['totalProfit'].iloc[0]
                end = timeit.timeit()
                print('profit = %.2f' % profit,'%.3fs' % (start-end))
    
    return stats.reset_index().drop('index', axis=1)

In [ ]:
stats = optimize([i / 10**4 for i in range(1, 11)], 
         [i / 10**4 for i in range(1, 11)], 
         [i / 10**4 for i in range(1, 11)])
write_tsv(stats, "custom-results.tsv")

In [ ]:
stats.head()

# Look on transactions

In [ ]:
len(strategy.exchange.transactions)

In [ ]:
winningTransactions = []
for i in range(int(len(strategy.exchange.transactions) / 2)):
    t1, t2 = strategy.exchange.transactions[i:i+2]
    if t1['amount'] < t2['amount']:
        winningTransactions += [t1, t2]

In [ ]:
len(winningTransactions)

In [ ]:
strategy.exchange.transactions[2:4]

In [ ]:
%matplotlib notebook
plot_transactions(ohlc_train, strategy.exchange.transactions[:])

for c in ['red', 'yellow', 'green', 'blue']:
    plt.plot(range(ohlc_train.shape[0]), ohlc_train[c], c=c)

In [ ]:
t = 70

transaction = strategy.exchange.transactions[t]
plotTransactionWithSurroundings(ohlc_train, strategy.historyLen, transaction)
transaction = strategy.exchange.transactions[t+1]
plotTransactionWithSurroundings(ohlc_train, strategy.historyLen, transaction)

In [ ]:
transaction

# Evaluate

In [ ]:
allStats = pd.read_csv('D:\\Dropbox\\My work\\krypl-project\\result\\customStrategy.tsv',sep='\t')

In [ ]:
allStats.query('totalProfit > 90 and winPercentage > 70').sort_values('numberOfTrades', ascending=False)

In [ ]:
allStats.sort_values('totalProfit', ascending=False).query('willingLoss < 0.1').query('winPercentage > 50')

In [ ]:
dataManager = deepcopy(data_test)
exchange = BackTestExchange(dataManager, deepcopy(wallet), 0.0025)
strategy = CustomStrategy(exchange, dataManager, 2016, contract_pair, 100, willingLoss=0.05,
                          distanceFromMaxThreshold1=0.06, distanceFromMaxThreshold2=0.03, targetProfit=0.01)
strategy.trade()
strategy.stats('usdt').report()